<a href="https://colab.research.google.com/github/raaz0000002/pytorch_learning/blob/main/NN_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import pandas as pd
import os

In [59]:
df = pd.read_csv("/content/sample_data/california_housing_train.csv")

In [60]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [61]:
df1 = pd.read_csv("/content/sample_data/california_housing_test.csv")

In [62]:
df.drop(columns = ["longitude","latitude"],inplace=True)

In [63]:
df

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...
16995,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [64]:
# Standard Scaler
# seperate X,Y AND CONVERT TO TENSOR WITH SAME DATA TYPE
from torch.utils.data import Dataset
import torch



In [65]:
from sklearn.preprocessing import StandardScaler

In [66]:
class HousingDataset(Dataset):
    def __init__(self, csv_file):
        # Load the CSV file
        df = pd.read_csv(csv_file)

        # Drop longitude and latitude columns
        df.drop(columns=["longitude", "latitude"], inplace=True)

        # Initialize the scaler
        self.scaler = StandardScaler()

        # Separate features (X) and target (y)
        self.x = df.drop(columns=["median_house_value"])
        self.y = df["median_house_value"]

        # Scale the features
        self.x = self.scaler.fit_transform(self.x)

        # Convert to PyTorch tensors
        self.x = torch.tensor(self.x, dtype=torch.float32)  # No .values needed after fit_transform
        self.y = torch.tensor(self.y.values, dtype=torch.float32)

    def __getitem__(self, index):
        # Return a tuple of (features, target) for the given index
        return self.x[index], self.y[index]

    def __len__(self):
        # Return the total number of samples
        return len(self.y)





In [67]:
housingdataset_train =  HousingDataset("/content/sample_data/california_housing_train.csv")

In [68]:
housingdataset_test =  HousingDataset("/content/sample_data/california_housing_test.csv")

In [69]:
test_dataset , validation_dataset = torch.utils.data.random_split(housingdataset_test,[1500,1500])   # kati parcentage vanera calclate garera rakhda ni hunxa

In [70]:
# creating dataloader
from torch.utils.data import DataLoader

In [71]:
train_dataloader = DataLoader(housingdataset_train,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=True)
validation_dataloader = DataLoader(validation_dataset,batch_size=32,shuffle=True)

In [72]:
test_dataset[0]

(tensor([ 1.8445, -0.2179, -0.1803, -0.4676, -0.2598, -0.4567]),
 tensor(500001.))

In [73]:
for batch_x,batch_y in train_dataloader:
  print(batch_x)
  print(batch_y)
  print(batch_x.shape)
  print(batch_y.shape)
  break

tensor([[ 1.1449e+00, -4.8611e-01, -1.4807e-01, -5.4937e-01, -1.7743e-01,
          6.0197e-01],
        [ 7.4767e-01, -5.4575e-01, -5.1345e-01, -2.8887e-01, -5.3112e-01,
         -1.2435e+00],
        [-1.2627e-01,  1.3470e+00,  3.4867e+00,  2.3832e+00,  3.2815e+00,
         -7.5786e-01],
        [ 1.3038e+00, -3.7877e-01, -3.1178e-01,  2.3996e-01, -3.3347e-01,
          8.4127e-02],
        [-8.4132e-01,  2.6072e-01,  6.1430e-03,  3.3753e-01,  1.7628e-02,
         -2.6428e-01],
        [-1.3975e+00,  5.0156e-01,  5.1624e-01,  6.8166e-01,  5.8718e-01,
         -4.0741e-01],
        [ 5.8877e-01, -2.2050e-01, -4.0905e-01, -4.4831e-01, -3.6728e-01,
          4.3772e-01],
        [-9.2077e-01, -1.7967e-01, -3.8296e-01,  3.0181e-01, -3.5427e-01,
          3.9454e-01],
        [-2.8517e-01, -3.9390e-01, -6.9139e-01, -5.9206e-01, -6.5076e-01,
          1.5498e+00],
        [ 2.7098e-01, -1.5949e-01,  4.8849e-02, -4.6675e-02,  8.0045e-02,
         -8.0330e-02],
        [ 9.8602e-01,  9.7875e

In [90]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 64)  # Input: 6 features, Output: 64 units
        self.act1 = nn.ReLU()        # Activation: ReLU
        self.fc2 = nn.Linear(64, 32) # Input: 64 units, Output: 32 units
        self.act2 = nn.ReLU()        # Activation: ReLU
        self.fc3 = nn.Linear(32, 1)  # Input: 32 units, Output: 1 (regression)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.fc3(x)  # No need for separate 'y', just return x
        return x




In [96]:
# Training the data,
#define optimizer and loss function
optim = Adam(model.parameters(),lr=0.001)
loss_fn = MSELoss()


In [92]:
from torch.optim import Adam
from torch.nn import MSELoss

In [93]:
num_epoch = 100

In [94]:
model = NeuralNet()

In [97]:
model

NeuralNet(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (act1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (act2): ReLU()
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

In [105]:
loss = 0

In [106]:
len(test_dataloader)

47

In [111]:
# Training and Validation Loop
for epoch in range(num_epoch):
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in train_dataloader:
        y_pred = model(batch_x)
        loss = loss_fn(y_pred, batch_y.unsqueeze(1))
        optim.zero_grad()
        loss.backward()
        optim.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epoch}, Training Loss: {avg_train_loss:.4f}")

    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in validation_dataloader:
            y_pred = model(batch_x)
            loss = loss_fn(y_pred, batch_y.unsqueeze(1))
            total_loss += loss.item()

    test_loss = total_loss / len(validation_dataloader)
    print(f"Epoch {epoch+1}/{num_epoch}, Validation Loss: {test_loss:.4f}")






Epoch 1/100, Training Loss: 10545721481.1429
Epoch 1/100, Validation Loss: 10252267340.2553
Epoch 2/100, Training Loss: 9809308235.0677
Epoch 2/100, Validation Loss: 9519596794.5532
Epoch 3/100, Training Loss: 9064958815.7594
Epoch 3/100, Validation Loss: 8882555310.2979
Epoch 4/100, Training Loss: 8436241646.6767
Epoch 4/100, Validation Loss: 8283422600.1702
Epoch 5/100, Training Loss: 7893323251.4887
Epoch 5/100, Validation Loss: 7764989614.2979
Epoch 6/100, Training Loss: 7416911756.2707
Epoch 6/100, Validation Loss: 7293008694.4681
Epoch 7/100, Training Loss: 7010246410.5865
Epoch 7/100, Validation Loss: 6914571884.9362
Epoch 8/100, Training Loss: 6667996574.5564
Epoch 8/100, Validation Loss: 6578105082.5532
Epoch 9/100, Training Loss: 6441964108.0301
Epoch 9/100, Validation Loss: 6326661305.1915
Epoch 10/100, Training Loss: 6219215921.0827
Epoch 10/100, Validation Loss: 6158519693.6170
Epoch 11/100, Training Loss: 6088023629.7143
Epoch 11/100, Validation Loss: 6039801518.2979
Epoc